# Final Project: SQL part

The coronavirus took the entire world by surprise, changing everyone's daily routine. City dwellers no longer spent their free time outside, going to cafes and malls; more people were home, reading books. That attracted the attention of startups that rushed to develop new apps for book lovers.
You've been given a database of one of the services competing in this market. It contains data on books, publishers, authors, and customer ratings and reviews of books. This information will be used to generate a value proposition for a new product.

## Goals of the study (tasks):

1. Find the number of books released after January 1, 2000.
2. Find the number of user reviews and the average rating for each book.
3. Identify the publisher that has released the greatest number of books with more than 50 pages (this will help you exclude brochures and similar publications from your analysis).
4. Identify the author with the highest average book rating: look only at books with at least 50 ratings.
5. Find the average number of text reviews among users who rated more than 50 books.

## Plan of work:

1. Connect to the database and [download the data](#part1)
2. Create [SQL queries for each task](#part2)
3. Output the [queries results](#part3)
4. Write [conclusions](#part4)

### Connecting to the database:

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
#!/usr/bin/python
db_config = {'user': '*****',
             'pwd': '*****',
             'host': '*****',
             'port': *****,
             'db': '*****'}

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

<a id='part1'></a>
### Downloading the data

In [3]:
query_books = ''' SELECT * FROM books
        '''
books = pd.io.sql.read_sql(query_books, con = engine, index_col = 'book_id')

query_authors = ''' SELECT * FROM authors
        '''
authors = pd.io.sql.read_sql(query_authors, con = engine, index_col = 'author_id')

query_publishers = ''' SELECT * FROM publishers
        '''
publishers = pd.io.sql.read_sql(query_publishers, con = engine, index_col = 'publisher_id')

query_ratings = ''' SELECT * FROM ratings
        '''
ratings = pd.io.sql.read_sql(query_ratings, con = engine, index_col = 'rating_id')

query_reviews = ''' SELECT * FROM reviews
        '''
reviews = pd.io.sql.read_sql(query_reviews, con = engine, index_col = 'review_id')

In [4]:
dfs = [books, authors, publishers, ratings, reviews]

In [5]:
for i in dfs:
    display(i.head(5))

,author_id,title,num_pages,publication_date,publisher_id
book_id,,,,,
1,546,'Salem's Lot,594,2005-11-01,93
2,465,1 000 Places to See Before You Die,992,2003-05-22,336
3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
5,125,1776,386,2006-07-04,268


,author
author_id,
1,A.S. Byatt
2,Aesop/Laura Harris/Laura Gibbs
3,Agatha Christie
4,Alan Brennert
5,Alan Moore/David Lloyd


,publisher
publisher_id,
1,Ace
2,Ace Book
3,Ace Books
4,Ace Hardcover
5,Addison Wesley Publishing Company


,book_id,username,rating
rating_id,,,
1,1,ryanfranco,4
2,1,grantpatricia,2
3,1,brandtandrea,5
4,2,lorichen,3
5,2,mariokeller,2


,book_id,username,text
review_id,,,
1,1,brandtandrea,Mention society tell send professor analysis. ...
2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
4,3,johnsonamanda,Finally month interesting blue could nature cu...
5,3,scotttamara,Nation purpose heavy give wait song will. List...


<a id='part2'></a>
### Creating SQL queries for tasks:

#### Find the number of books released after January 1, 2000.

In [6]:
query_1 = ''' SELECT COUNT(books.*) AS n_books
            FROM books
            WHERE books.publication_date > '2000-01-01';
        '''

#### Find the number of user reviews and the average rating for each book.

In [7]:
query_2 = ''' SELECT
                ratings.book_id AS book_id,
                AVG(ratings.rating) AS avg_rating,
                COUNT(reviews.*) AS n_reviews
            FROM
                ratings INNER JOIN
                (SELECT
                    reviews.book_id AS book_id,
                    COUNT(reviews.*) AS reviews_number
                FROM
                    reviews
                GROUP BY 
                    reviews.book_id
                ) reviews ON ratings.book_id = reviews.book_id
            GROUP BY 
                ratings.book_id
            ORDER BY
                book_id;
        '''

#### Identify the publisher that has released the greatest number of books with more than 50 pages.

In [8]:
query_3 = ''' SELECT
                COUNT (books.*) AS books_released,
                publishers.publisher AS publisher
            FROM
                books
            INNER JOIN publishers ON books.publisher_id = publishers.publisher_id
            WHERE
                books.num_pages > 50
            GROUP BY
                books.publisher_id,
                publishers.publisher
            ORDER BY
                COUNT(BOOKS.*) DESC
            LIMIT 1;
        '''

#### Identify the author with the highest average book rating: look only at books with at least 50 ratings.

In [9]:
query_4 = '''  SELECT
                books.author_id AS author_id,
                authors.author AS author,
                AVG(ratings.rating) AS avg_rating
            FROM
                books INNER JOIN
                (SELECT
                    books.book_id AS book_id,
                    books.author_id AS author_id,
                    COUNT(ratings.rating) AS n_ratings
                FROM
                    books
                INNER JOIN ratings ON books.book_id = ratings.book_id
                GROUP BY
                    books.book_id,
                    books.author_id
                HAVING
                    COUNT(ratings.rating) > 50
                    ) AS sub ON books.author_id = sub.author_id
            INNER JOIN ratings ON books.book_id = ratings.book_id
            INNER JOIN authors ON books.author_id = authors.author_id
            GROUP BY
                books.author_id,
                authors.author
            ORDER BY
                AVG(ratings.rating) DESC
            LIMIT 1;
        '''

#### Find the average number of text reviews among users who rated more than 50 books.

In [10]:
query_5 = ''' SELECT
                AVG(sub.n_reviews) AS avg_reviews
            FROM
                (SELECT
                    ratings.username AS user,
                    COUNT(ratings.book_id) AS books_rated,
                    sub.n_reviews AS n_reviews
                FROM
                    ratings INNER JOIN
                    (SELECT
                    reviews.username AS user,
                    COUNT(reviews.text) AS n_reviews
                    FROM
                        reviews
                    GROUP BY
                        reviews.username
                    ) AS sub ON ratings.username = sub.user
                GROUP BY
                    ratings.username,
                    sub.n_reviews
                HAVING
                    COUNT(ratings.book_id) > 50
                ) AS sub;
        '''

<a id='part3'></a>
### Outputting the query results:

In [11]:
q1 = pd.io.sql.read_sql(query_1, con = engine)

display(q1)
print()
print('The number of books released after January 1, 2000: {} books.'.format(q1['n_books'].iloc[0]))

,n_books
0,819



The number of books released after January 1, 2000: 819 books.


In [12]:
q2 = pd.io.sql.read_sql(query_2, con = engine)

print('Number of user reviews and the average rating for each book:')
display(q2)
q2[['avg_rating', 'n_reviews']].describe()

Number of user reviews and the average rating for each book:


,book_id,avg_rating,n_reviews
0,1,3.666667,3
1,2,2.500000,2
2,3,4.666667,3
3,4,4.500000,2
4,5,4.000000,6
...,...,...,...
989,996,3.666667,3
990,997,3.400000,5
991,998,3.200000,5
992,999,4.500000,2


,avg_rating,n_reviews
count,994.000000,994.000000
mean,3.898699,6.481891
std,0.562190,11.777997
min,1.500000,2.000000
25%,3.500000,2.000000
50%,4.000000,3.000000
75%,4.333333,5.000000
max,5.000000,160.000000


In [13]:
q3 = pd.io.sql.read_sql(query_3, con = engine)

display(q3)
print()
print('Publisher that has released the greatest number of books with more than 50 pages: {} publisher with {} books.'.format
      ((q3['publisher'].iloc[0]), (q3['books_released'].iloc[0])))

,books_released,publisher
0,42,Penguin Books



Publisher that has released the greatest number of books with more than 50 pages: Penguin Books publisher with 42 books.


In [14]:
q4 = pd.io.sql.read_sql(query_4, con = engine)

display(q4)
print()
print('The author with the highest average book rating (among books with at least 50 ratings) is {} with average rating {:.2f}'.format
     ((q4['author'].iloc[0]), (q4['avg_rating'].iloc[0])))

,author_id,author,avg_rating
0,236,J.K. Rowling/Mary GrandPré,4.288462



The author with the highest average book rating (among books with at least 50 ratings) is J.K. Rowling/Mary GrandPré with average rating 4.29


In [15]:
q5 = pd.io.sql.read_sql(query_5, con = engine)

display(q5)
print()
print('The average number of text reviews (among users who rated more than 50 books): {:.2f}'.format(q5['avg_reviews'].iloc[0]))

,avg_reviews
0,24.333333



The average number of text reviews (among users who rated more than 50 books): 24.33


<a id='part4'></a>
### Conclusions:

In our database we have data on 819 books released after January 1, 2000, which is not a very big number. <br>
These books have rating between 1.5 and 5 with average rating of 3.89. Also these books have from 2 to 160 reviews, with 6.5 reviews on average. <br>
Let's take a look only at books with more than 50 pages. Their biggest publisher is Penguin Books, which has published 42 books. <br>
If we look at very popular books (books with more than 50 ratings), we see that the author with the highest average rating is J.K. Rowling (Mary GrandPré	is an illustrator) with average book rating of 4.28 <br>
Finally, if we look at super-active users (users who rated more than 50 books), we see that on average these super-active users write 24.3 text reviews (meaning on average little less than a half of their ratings is followed by a text review).